<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test

In [ ]:
from torch import nn
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict, Audio
from model.pure_model import Wave_Network
from torch.utils.data import DataLoader
import evaluate
accuracy = evaluate.load("accuracy")
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import load_from_disk, Audio
import matplotlib.pyplot as plt

dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

def edit_label(examples):
    # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
    for i, x in enumerate(examples['file']):
        a = examples['label'][i]
        if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
        elif id2label[str(examples['label'][i])] == '_silence_':
            examples['label'][i] = dict_label['silence']
    return examples

if __name__ == "__main__":
    model_path = './model_1'
    weight_state_dict = r'D:\1.Project\3.Machine_Learning\Voice\W2Vec\models\w2vec_final.pth'
    epoch_num = 30
    batch_size = 32

    label2id = {'yes': '0', 'no': '1', 'up': '2', 'down': '3', 'left': '4', 'right': '5', 'on': '6', 'off': '7', 'stop': '8', 'go': '9', 'unknown': '10', 'silence': '11'}
    id2label = {'0':'yes', '1':'no', '2':'up', '3':'down', '4':'left', '5':'right', '6':'on', '7':'off', '8':'stop', '9':'go', '10':'unknown', '11':'silence'}

    train_data = load_from_disk('./anomaly/data/dataset/train_update')
    valid_data = load_from_disk('./anomaly/data/dataset/validation_update')
    test_dataset = load_from_disk('./anomaly/data/dataset/test_update')

    print(train_data)

    train_dataloader = DataLoader(train_data.with_format('torch'), batch_size, shuffle=True)
    test_dataloader = DataLoader(valid_data.with_format('torch'), batch_size, shuffle=True)

    num_class = len(label2id) - 1
    model = Wave_Network(num_classes=num_class, model_path=model_path)
    # model.load_state_dict(torch.load(weight_state_dict))

    criterion_1 = nn.CrossEntropyLoss()
    criterion_2 = nn.BCEWithLogitsLoss()
    # optimizer = torch.optim.Adam(model.get_train_params(), lr=0.001)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epoch_num):
        loss_item = 0.
        valid_loss = 0.
        running_loss = 0
        save_loss = 999999
        loss_item_1 = 0.
        loss_item_2 = 0.
        model.train()
        for i, data in enumerate(train_dataloader):
            optimizer.zero_grad()
            inputs, labels = data['input_ids'].cuda(), data['anomaly_label'].cuda()
            normal_inputs = data['input_ids'][data['anomaly_label']==1].cuda()
            anormal_inputs = data['input_ids'][data['anomaly_label']==0].cuda()
            normal_labels = data['re_label'][data['anomaly_label']==1].cuda()

            if len(normal_inputs) == 0: continue

            # true_project_label = torch.ones(size=(normal_inputs.shape[0], 1))
            original_project_label = torch.unsqueeze(data['anomaly_label']==1, dim=-1)
            false_project_label = torch.zeros(size=(normal_inputs.shape[0], 1))
            projection_labels = torch.concat((original_project_label, false_project_label)).cuda()
            o_nomaly, o_classification = model(inputs, labels)
            loss_1 = criterion_1(o_classification, normal_labels)
            loss_2 = criterion_2(o_nomaly, projection_labels)
            loss = loss_1 + loss_2

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            loss_item_1 += loss_1.item()
            loss_item_2 += loss_2.item()
            if i % 10 == 9:  # Print every 10 batches
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] running_loss: {running_loss/10} loss_cls: {loss_item_1/10} loss_anomaly: {loss_item_2/10}')
                running_loss = 0.0
                loss_item_1 = 0.0
                loss_item_2 = 0.0

        model.eval()
        with torch.no_grad():
            for i, data in enumerate(test_dataloader):
                inputs, labels = data['input_ids'].cuda(), data['anomaly_label'].cuda()
                normal_inputs = data['input_ids'][data['anomaly_label']==1].cuda()
                anormal_inputs = data['input_ids'][data['anomaly_label']==0].cuda()
                normal_labels = data['re_label'][data['anomaly_label']==1].cuda()
                if len(normal_inputs) == 0: continue
                # true_project_label = torch.ones(size=(normal_inputs.shape[0], 1))
                original_project_label = torch.unsqueeze(data['anomaly_label']==1, dim=-1)
                false_project_label = torch.zeros(size=(normal_inputs.shape[0], 1))
                projection_labels = torch.concat((original_project_label, false_project_label)).cuda()
                o_nomaly, o_classification = model(inputs, labels)
                loss = criterion_1(o_classification, normal_labels) + criterion_2(o_nomaly, projection_labels)
                valid_loss += loss.item()
            valid_loss = valid_loss/len(test_dataloader)
            if epoch == 0: valid_loss = save_loss
            else:
                if valid_loss < save_loss:
                    torch.save(model.state_dict(), f'./w2vec/models/w2vec_best.pth')
                    save_loss = valid_loss
            print(f"Epoch [{epoch+1}/{epoch_num}], Valid Loss: {valid_loss:.4f}")

    torch.save(model.state_dict(), f'./w2vec/models/w2vec_final.pth')


In [ ]:
from torch import nn
from transformers import Wav2Vec2Model, AutoModelForAudioClassification
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict
import torch
import torch.nn.functional as F


def init_weight(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
    elif isinstance(m, torch.nn.Conv2d):
        torch.nn.init.xavier_normal_(m.weight)

class Projection(torch.nn.Module):

    def __init__(self, in_planes, out_planes=None, n_layers=1, layer_type=0):
        super(Projection, self).__init__()

        if out_planes is None:
            out_planes = in_planes
        self.layers = torch.nn.Sequential()
        _in = None
        _out = None
        for i in range(n_layers):
            _in = in_planes if i == 0 else _out
            _out = out_planes
            self.layers.add_module(f"{i}fc",
                                   torch.nn.Linear(_in, _out))
            if i < n_layers - 1:
                # if layer_type > 0:
                #     self.layers.add_module(f"{i}bn",
                #                            torch.nn.BatchNorm1d(_out))
                if layer_type > 1:
                    self.layers.add_module(f"{i}relu",
                                           torch.nn.LeakyReLU(.2))
        self.apply(init_weight)

    def forward(self, x):

        # x = .1 * self.layers(x) + x
        x = self.layers(x)
        return x

class Discriminator(torch.nn.Module):
    def __init__(self, in_planes, n_layers=3, hidden=None):
        super(Discriminator, self).__init__()

        _hidden = in_planes if hidden is None else hidden
        self.body = torch.nn.Sequential()
        for i in range(n_layers-1):
            _in = in_planes if i == 0 else _hidden
            _hidden = int(_hidden // 1.5) if hidden is None else hidden
            self.body.add_module('block%d'%(i+1),
                                 torch.nn.Sequential(
                                     torch.nn.Linear(_in, _hidden),
                                     torch.nn.BatchNorm1d(_hidden),
                                     torch.nn.LeakyReLU(0.2)
                                 ))
        self.tail = torch.nn.Linear(_hidden, 1, bias=False)
        self.apply(init_weight)

    def forward(self,x):
        x = self.body(x)
        x = self.tail(x)
        return x

class Wave_Network(nn.Module):
    def __init__(self, num_classes=12, model_path='', device='cuda'):
        super().__init__()
        self.device = device
        self.backbone = Wav2Vec2Model.from_pretrained(model_path).to(device=device)

        for param in self.backbone.parameters():
            param.requires_grad = False
        self.backbone.feature_extractor._freeze_parameters()

        self.discriminator = Discriminator(768).to(device=device)
        self.projection = Projection(768, 768, 3).to(device=device)
        self.projector = nn.LazyLinear(256).to(device=device)
        self.classifier = nn.Sequential(
            # nn.Flatten(),
            # nn.LazyLinear(256).to(device=device),
            nn.LazyLinear(num_classes).to(device=device)
        )

    def get_train_params(self):
        return [
            {'params': self.projection.parameters()},
            {'params': self.discriminator.parameters()},
            {'params': self.projector.parameters()},
            {'params': self.classifier.parameters()},
        ]

    def forward(self, x, anomaly_label=None, std=0.05, is_train=True):
        x = self.backbone(x).last_hidden_state
        x_reshape = x.reshape(x.shape[0],-1)
        x_hidden= F.adaptive_avg_pool1d(x_reshape, 768)
        x_hidden_state = self.projector(x)
        x_hidden_state = x_hidden_state.mean(dim=1)

        if is_train:
            normal_hidden_state = x_hidden_state[anomaly_label==1]
            x_classification = self.classifier(normal_hidden_state)

            x_projector = self.projection(x_hidden)
            # add noise
            # if is_train:
            normal_x_projector = x_projector[anomaly_label==1]
            noise = torch.normal(mean=0, std=std, size=normal_x_projector.shape).to(device=self.device)
            x_noise = normal_x_projector + noise
            x_noise = torch.concat((x_projector, x_noise))
            # else:
            #     x_noise = x_projector
            x_anomaly = self.discriminator(x_noise)
        else:
            x_classification = self.classifier(x_hidden_state)
            x_projector = self.projection(x_hidden)
            x_anomaly = self.discriminator(x_projector)
        return x_anomaly, x_classification

    def save(self, file_path):
        # Custom logic before saving
        print(f"Saving model to {file_path}")

        # For example, saving both the model's state dict and some metadata
        torch.save({
            'model_state_dict_backbone': self.backbone.state_dict(),
            'model_state_dict_discriminator': self.discriminator.state_dict(),
            'model_state_dict_projection': self.projection.state_dict(),
            'model_state_dict_classifier': self.classifier.state_dict(),
            'custom_metadata': {
                'info': 'This is a custom saved model',
                'epoch': 10,
                'loss': 0.1234
            }
        }, file_path)

        # Custom logic after saving
        print("Model saved successfully!")

    def load(self, file_path):
        # Custom logic before saving
        print(f"Loading model from {file_path}")

        # For example, saving both the model's state dict and some metadata
        temp_model = torch.load(file_path)
        self.backbone.load_state_dict(temp_model['model_state_dict_backbone'])
        self.discriminator.load_state_dict(temp_model['model_state_dict_discriminator'])
        self.projection.load_state_dict(temp_model['model_state_dict_projection'])
        self.classifier.load_state_dict(temp_model['model_state_dict_classifier'])
        # Custom logic after saving
        print("Model loaded successfully!")

# _network =  Wav2Vec2Model.from_pretrained(pretrained_model_name_or_path='./model_1')
# print(_network)
# for name, param in _network.named_parameters():
#     print(f"layer: {name} | Shape: {param.shape}")

# model = AutoModelForAudioClassification.from_pretrained(
#         pretrained_model_name_or_path='./model_1', num_labels=10, ignore_mismatched_sizes=True)
# for name, param in model.named_parameters():
#     print(f"layer: {name} | Shape: {param.shape}")

In [ ]:
from datasets import load_from_disk, Audio
import numpy as np

dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}
def collate_fn(seq):
    # if (len(seq['audio']['array']) < 16_000):
    #     seq['audio']['array'] = np.pad(seq['audio']['array'], (0, 16_000 - len(seq['audio']['array'])), mode='constant', constant_values=0)
    # else:
    #     seq['audio']['array'] = seq['audio']['array'][:16_000]
    if (len(seq['audio']['array']) < 16_000):
        seq['input_ids'] = np.pad(seq['audio']['array'], (0, 16_000 - len(seq['audio']['array'])), mode='constant', constant_values=0)
    else:
        seq['input_ids'] = seq['audio']['array'][:16_000]

    if seq['label'] == 10:
        seq['anomaly_label'] = 0
        seq['re_label'] = -1
    else:
        seq['anomaly_label'] = 1
        if seq['label'] == 11:
            seq['re_label'] = 10
        else:
            seq['re_label'] = seq['label']
    return seq

train_data = load_from_disk('./anomaly/data/dataset/train')
valid_data = load_from_disk('./anomaly/data/dataset/validation')
test_dataset = load_from_disk('./anomaly/data/dataset/test')
# padding
train_data = train_data.map(collate_fn, remove_columns="audio")
valid_data = valid_data.map(collate_fn, remove_columns="audio")
test_dataset = test_dataset.map(collate_fn, remove_columns="audio")
print(train_data)
print(valid_data)
print(test_dataset)
train_data.save_to_disk('./anomaly/data/dataset/train_update')
valid_data.save_to_disk('./anomaly/data/dataset/validation_update')
test_dataset.save_to_disk('./anomaly/data/dataset/test_update')

# train_data = load_dataset('./anomaly/data/dataset/train')
# valid_data = load_dataset('./anomaly/data/dataset/validation')
# test_dataset = load_dataset('./anomaly/data/dataset/test')
print("Done ~")

In [ ]:
from torch import nn
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict, Audio
from model.pure_model import Wave_Network
from torch.utils.data import DataLoader
import evaluate
accuracy = evaluate.load("accuracy")
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import load_from_disk, Audio
import matplotlib.pyplot as plt

dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

def edit_label(examples):
    # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
    for i, x in enumerate(examples['file']):
        a = examples['label'][i]
        if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
        elif id2label[str(examples['label'][i])] == '_silence_':
            examples['label'][i] = dict_label['silence']
    return examples

if __name__ == "__main__":
    model_path = './model_1'
    weight_state_dict = r'D:\1.Project\3.Machine_Learning\Voice\W2Vec\models\w2vec_final.pth'
    epoch_num = 30
    batch_size = 1

    label2id = {'yes': '0', 'no': '1', 'up': '2', 'down': '3', 'left': '4', 'right': '5', 'on': '6', 'off': '7', 'stop': '8', 'go': '9', 'unknown': '10', 'silence': '11'}
    id2label = {'0':'yes', '1':'no', '2':'up', '3':'down', '4':'left', '5':'right', '6':'on', '7':'off', '8':'stop', '9':'go', '10':'unknown', '11':'silence'}

    test_dataset = load_from_disk('./anomaly/data/dataset/test_update')

    print(test_dataset)

    test_dataset = DataLoader(test_dataset.with_format('torch'), batch_size, shuffle=True)

    num_class = len(label2id) - 1
    model = Wave_Network(num_classes=num_class, model_path=model_path)
    model.load_state_dict(torch.load(weight_state_dict))

    for epoch in range(epoch_num):
        loss_item = 0.
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(test_dataset):
                inputs = data['input_ids'].cuda()
                o_nomaly, o_classification = model(inputs, is_train=False)
                anomaly_arg = torch.sigmoid(o_nomaly)
                cls_arg = torch.argmax(o_classification, dim=-1)
                print(anomaly_arg, data['anomaly_label'], cls_arg, data['re_label'])


In [ ]:
from transformers import AutoFeatureExtractor
from datasets import load_dataset, Audio
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")


import pandas as pd
import matplotlib.pyplot as plt

dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16_000, truncation=True
    )
    return inputs


def edit_label(examples):
    # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
    for i, x in enumerate(examples['file']):
        a = examples['label'][i]
        if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
        elif id2label[str(examples['label'][i])] == '_silence_':
            examples['label'][i] = dict_label['silence']
    return examples

feature_extractor = AutoFeatureExtractor.from_pretrained('./model_1')
train_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='train', trust_remote_code=True)
valid_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='validation', trust_remote_code=True)
test_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='test', trust_remote_code=True)

labels = valid_dataset_.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label


train_dataset_ = train_dataset_.map(edit_label, batched=True)
valid_dataset_ = valid_dataset_.map(edit_label, batched=True)
test_dataset_ = test_dataset_.map(edit_label, batched=True)

df = train_dataset_.to_pandas()
# Giả sử cột label trong dataset có tên là 'label'
label_counts = df['label'].value_counts()

# Hiển thị kết quả
print(label_counts)

# # Hiển thị đồ thị
# label_counts.plot(kind='bar')
# plt.xlabel('Class')
# plt.ylabel('Số lượng')
# plt.title('Số lượng mỗi class trong cột label')
# plt.show()

train_dataset_ = train_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])
valid_dataset_ = valid_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])
test_dataset_ = test_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])

labels = dict_label.keys()
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
print(label2id)



train_data = train_dataset_.cast_column("audio", Audio(sampling_rate=16_000))
valid_data = valid_dataset_.cast_column("audio", Audio(sampling_rate=16_000))
test_dataset = test_dataset_.cast_column("audio", Audio(sampling_rate=16_000))

encoded_data_train = train_data.map(preprocess_function, remove_columns="audio", batched=True)
encoded_data_validation = valid_data.map(preprocess_function, remove_columns="audio", batched=True)
encoded_test_validation = test_dataset.map(preprocess_function, remove_columns="audio", batched=True)
print(encoded_data_train)
print(encoded_data_validation)
print(encoded_test_validation)

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    # "./model_1", num_labels=num_labels, label2id=label2id, id2label=id2label
    "./my_awesome_mind_model/checkpoint-1597", num_labels=num_labels, label2id=label2id, id2label=id2label
)
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to='none',
    save_total_limit=3,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data_train.with_format("torch"),
    eval_dataset=encoded_data_validation.with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)
# trainer.train()
eval_results = trainer.evaluate(eval_dataset=encoded_test_validation.with_format("torch"))

print(eval_results)
print(trainer.log_metrics("eval", eval_results))


In [ ]:
from torch import nn
from transformers import Wav2Vec2Model, AutoModelForAudioClassification
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict
import torch
import torch.nn.functional as F


def init_weight(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
    if isinstance(m, torch.nn.BatchNorm2d):
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    elif isinstance(m, torch.nn.Conv2d):
        m.weight.data.normal_(0.0, 0.02)


class Projection(torch.nn.Module):

    def __init__(self, in_planes, out_planes=None, n_layers=1, layer_type=0):
        super(Projection, self).__init__()

        if out_planes is None:
            out_planes = in_planes
        self.layers = torch.nn.Sequential()
        _in = None
        _out = None
        for i in range(n_layers):
            _in = in_planes if i == 0 else _out
            _out = out_planes
            self.layers.add_module(f"{i}fc",
                                   torch.nn.Linear(_in, _out))
            self.layers.add_module(f"{i}bn",
                                    torch.nn.BatchNorm1d(_out))
            self.layers.add_module(f"{i}relu",
                                           torch.nn.LeakyReLU(.2))
            # if i < n_layers - 1:
            #     # if layer_type > 0:
            #     #     self.layers.add_module(f"{i}bn",
            #     #                            torch.nn.BatchNorm1d(_out))
            #     if layer_type > 1:
            #         self.layers.add_module(f"{i}relu",
            #                                torch.nn.LeakyReLU(.2))
        self.apply(init_weight)

    def forward(self, x):

        # x = .1 * self.layers(x) + x
        x = self.layers(x)
        return x

class Discriminator(torch.nn.Module):
    def __init__(self, in_planes, n_layers=3, hidden=None):
        super(Discriminator, self).__init__()

        _hidden = in_planes if hidden is None else hidden
        self.body = torch.nn.Sequential()
        for i in range(n_layers-1):
            _in = in_planes if i == 0 else _hidden
            _hidden = int(_hidden // 1.5) if hidden is None else hidden
            self.body.add_module('block%d'%(i+1),
                                 torch.nn.Sequential(
                                     nn.Dropout(0.2),
                                     torch.nn.Linear(_in, _hidden),
                                     torch.nn.BatchNorm1d(_hidden),
                                     torch.nn.LeakyReLU(0.2)
                                 ))
        self.tail = torch.nn.Linear(_hidden, 1, bias=False)
        self.apply(init_weight)

    def forward(self,x):
        x = self.body(x)
        x = self.tail(x)
        return x

class Wave_Network(nn.Module):
    def __init__(self, num_classes=12, model_path='', device='cuda'):
        super().__init__()
        self.device = device
        self.backbone = Wav2Vec2Model.from_pretrained(model_path).to(device=device)

        for param in self.backbone.parameters():
            param.requires_grad = False
        self.backbone.feature_extractor._freeze_parameters()

        self.discriminator = Discriminator(768).to(device=device)
        self.projection = Projection(768, 768, 2).to(device=device)
        self.projector = nn.Sequential(
            nn.Dropout(0.3),
            nn.LazyLinear(256).to(device=device)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            # nn.Flatten(),
            # nn.LazyLinear(256).to(device=device),
            nn.LazyLinear(num_classes).to(device=device)
        )

    def get_train_params(self, is_classification=True):
        if is_classification:
            return [
                {'params': self.projector.parameters()},
                {'params': self.classifier.parameters()},
            ]
        else:
            return [
                {'params': self.projection.parameters()},
                {'params': self.discriminator.parameters()},
            ]

    def forward(self, x, anomaly_label=None, std=0.1, is_train=True):
        x = self.backbone(x).last_hidden_state
        x_reshape = x.reshape(x.shape[0],-1)
        x_hidden= F.adaptive_avg_pool1d(x_reshape, 768)
        x_hidden_state = self.projector(x)
        x_hidden_state = x_hidden_state.mean(dim=1)

        if is_train:
            normal_hidden_state = x_hidden_state[anomaly_label==1]
            x_classification = self.classifier(normal_hidden_state)

            x_projector = self.projection(x_hidden)
            # add noise
            # if is_train:
            normal_x_projector = x_projector[anomaly_label==1]
            noise = torch.normal(mean=0, std=std, size=normal_x_projector.shape).to(device=self.device)
            x_noise = normal_x_projector + noise
            x_noise = torch.concat((x_projector, x_noise))
            # else:
            #     x_noise = x_projector
            x_anomaly = self.discriminator(x_noise)
        else:
            x_classification = self.classifier(x_hidden_state)
            x_projector = self.projection(x_hidden)
            x_anomaly = self.discriminator(x_projector)
        return x_anomaly, x_classification

    def save(self, file_path):
        # Custom logic before saving
        print(f"Saving model to {file_path}")

        # For example, saving both the model's state dict and some metadata
        torch.save({
            'model_state_dict_backbone': self.backbone.state_dict(),
            'model_state_dict_discriminator': self.discriminator.state_dict(),
            'model_state_dict_projection': self.projection.state_dict(),
            'model_state_dict_classifier': self.classifier.state_dict(),
            'custom_metadata': {
                'info': 'This is a custom saved model',
                'epoch': 10,
                'loss': 0.1234
            }
        }, file_path)

        # Custom logic after saving
        print("Model saved successfully!")

    def load(self, file_path):
        # Custom logic before saving
        print(f"Loading model from {file_path}")

        # For example, saving both the model's state dict and some metadata
        temp_model = torch.load(file_path)
        self.backbone.load_state_dict(temp_model['model_state_dict_backbone'])
        self.discriminator.load_state_dict(temp_model['model_state_dict_discriminator'])
        self.projection.load_state_dict(temp_model['model_state_dict_projection'])
        self.classifier.load_state_dict(temp_model['model_state_dict_classifier'])
        # Custom logic after saving
        print("Model loaded successfully!")

# _network =  Wav2Vec2Model.from_pretrained(pretrained_model_name_or_path='./model_1')
# print(_network)
# for name, param in _network.named_parameters():
#     print(f"layer: {name} | Shape: {param.shape}")

# model = AutoModelForAudioClassification.from_pretrained(
#         pretrained_model_name_or_path='./model_1', num_labels=10, ignore_mismatched_sizes=True)
# for name, param in model.named_parameters():
#     print(f"layer: {name} | Shape: {param.shape}")

In [ ]:
from datasets import load_dataset, Audio
import numpy as np

dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}
def collate_fn(seq):

    seq['audio']['array'] = np.pad(seq, (0, 16_000 - len(seq['audio']['array'])), mode='constant', constant_values=0)
    return seq

def edit_label(examples):
    # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
    for i, x in enumerate(examples['file']):
        a = examples['label'][i]
        if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
        elif id2label[str(examples['label'][i])] == '_silence_':
            examples['label'][i] = dict_label['silence']
    return examples

# train_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='train', trust_remote_code=True)
valid_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='validation', trust_remote_code=True)
# test_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='test', trust_remote_code=True)

labels = valid_dataset_.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label


# train_dataset_ = train_dataset_.map(edit_label, batched=True)
valid_dataset_ = valid_dataset_.map(edit_label, batched=True)
# test_dataset_ = test_dataset_.map(edit_label, batched=True)

# df = train_dataset_.to_pandas()
# # Giả sử cột label trong dataset có tên là 'label'
# label_counts = df['label'].value_counts()

# # Hiển thị kết quả
# print(label_counts)

# # Hiển thị đồ thị
# label_counts.plot(kind='bar')
# plt.xlabel('Class')
# plt.ylabel('Số lượng')
# plt.title('Số lượng mỗi class trong cột label')
# plt.show()

# train_dataset_ = train_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])
valid_dataset_ = valid_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])
# test_dataset_ = test_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])



labels = dict_label.keys()
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
print(label2id)

# train_data = train_dataset_.cast_column("audio", Audio(sampling_rate=16_000))
valid_data = valid_dataset_.cast_column("audio", Audio(sampling_rate=16_000))
# test_dataset = test_dataset_.cast_column("audio", Audio(sampling_rate=16_000))

# padding
# train_dataset_ = train_dataset_.map(collate_fn)
valid_dataset_ = valid_dataset_.map(collate_fn)
# test_dataset_ = test_dataset_.map(collate_fn)

# train_data.save_to_disk('./anomaly/data/dataset/train')
valid_data.save_to_disk('./anomaly/data/dataset/validation')
# test_dataset.save_to_disk('./anomaly/data/dataset/test')

# train_data = load_dataset('./anomaly/data/dataset/train')
# valid_data = load_dataset('./anomaly/data/dataset/validation')
# test_dataset = load_dataset('./anomaly/data/dataset/test')
print("Done ~")